<a href="https://colab.research.google.com/github/visheshagrawal/Affective-Computing-IIITD/blob/main/AFC_A2_2018420_Vishesh_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
from torch.optim import lr_scheduler
from torchvision import datasets,transforms
import time
import os
import copy
%matplotlib inline

In [ ]:
%cd /content/drive/MyDrive/Academic/Sem8/Affective Computing/Assignment_2
%ls

/content/drive/MyDrive/Academic/Sem8/Affective Computing/Assignment_2
 AFC_A2_2018420.ipynb  'Assignment 2 data'@


In [ ]:
#DEVICE (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#Data Loading

In [ ]:
#Used to load images into tensor format and resized format
transform = transforms.Compose([transforms.Resize((300,300)),transforms.ToTensor() ])

In [ ]:
source_data_path="/content/drive/MyDrive/Academic/Sem8/Affective Computing/Assignment_2/Assignment 2 data/Source"
target_data_path="/content/drive/MyDrive/Academic/Sem8/Affective Computing/Assignment_2/Assignment 2 data/Target"

In [ ]:
source_dataset = datasets.ImageFolder(source_data_path,transform)

In [ ]:
target_dataset = datasets.ImageFolder(target_data_path,transform)

In [ ]:
#Splitting source domain into train and test
source_total_length = len(source_dataset)
source_train_size= int(0.8*source_total_length)
source_val_size= source_total_length - source_train_size

source_train_set, source_val_set = torch.utils.data.random_split(source_dataset, [source_train_size,source_val_size])

In [ ]:
#Splitting target domain intro train and test
target_total_length = len(target_dataset)
target_train_size= int(0.8*target_total_length)
target_val_size= target_total_length - target_train_size

target_train_set, target_val_set = torch.utils.data.random_split(target_dataset, [target_train_size,target_val_size])

In [ ]:
bs=8

In [ ]:
source_train_loader = torch.utils.data.DataLoader(source_train_set, batch_size=bs, shuffle=True)
source_val_loader = torch.utils.data.DataLoader(source_val_set, batch_size=bs, shuffle=True)

In [ ]:
target_train_loader = torch.utils.data.DataLoader(target_train_set, batch_size=bs, shuffle=True)
target_val_loader = torch.utils.data.DataLoader(target_val_set, batch_size=bs, shuffle=True)

In [ ]:
# source_loader = torch.utils.data.DataLoader(source_dataset, batch_size=32, shuffle=True)
# target_loader = torch.utils.data.DataLoader(target_dataset, batch_size=32, shuffle=True)

#Model

In [ ]:
# Defining the model
import torchvision.models as models
model_ft=models.vgg16(pretrained=True)
model_ft.classifier[6].out_features=7
model_ft = model_ft.to(device)
#Loss function is Cross Entropy
criterion = nn.CrossEntropyLoss()
learning_rate= 1e-4 
#Using Adam Optimizer
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=learning_rate)

In [ ]:
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
source_train_steps= len(source_train_set)//bs
source_val_steps=len(source_val_set)//bs

In [ ]:
target_train_steps= len(target_train_set)//bs
target_val_steps=len(target_val_set)//bs

In [ ]:
num_epochs=5

Training Model on Source Domain

In [ ]:
for e in range(num_epochs):
  model_ft.train()
  total_train_loss=0
  total_val_loss=0
  train_correct=0
  val_correct=0

  for (x,y) in source_train_loader:
    (x,y)=(x.to(device),y.to(device))
    pred=model_ft(x)
    loss=criterion(pred,y)
    optimizer_ft.zero_grad()
    loss.backward()
    optimizer_ft.step()
    total_train_loss+=loss
    train_correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
  model_ft.eval()
  for (x,y) in source_val_loader:
    (x,y)=(x.to(device),y.to(device))
    pred=model_ft(x)
    loss=criterion(pred,y)
    # optimizer_ft.zero_grad()
    # loss.backward()
    # optimizer_ft.step()
    total_val_loss+=loss
    val_correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
  
  avg_train_loss=total_train_loss/source_train_steps
  avg_val_loss=total_train_loss/source_val_steps
  train_correct = train_correct/len(source_train_set)
  val_correct = val_correct/len(source_val_set)

  print("Training Loss",avg_train_loss)
  print("Training Accuracy",train_correct)
  print("Val Loss",avg_val_loss)
  print("Val Accuracy",val_correct)



Training Loss tensor(0.0297, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.9928571428571429
Val Loss tensor(0.1300, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.7142857142857143
Training Loss tensor(0.0795, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.9785714285714285
Val Loss tensor(0.3477, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.7285714285714285
Training Loss tensor(0.0343, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.9857142857142858
Val Loss tensor(0.1500, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.6857142857142857
Training Loss tensor(0.1330, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.95
Val Loss tensor(0.5820, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.6857142857142857
Training Loss tensor(0.0431, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.9892857142857143
Val Loss tensor(0.1888, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.68571428571

In [ ]:
num_epochs=5

Training Model on Target Domain

In [ ]:
for e in range(num_epochs):
  model_ft.train()
  total_train_loss=0
  total_val_loss=0
  train_correct=0
  val_correct=0

  for (x,y) in target_train_loader:
    (x,y)=(x.to(device),y.to(device))
    pred=model_ft(x)
    loss=criterion(pred,y)
    optimizer_ft.zero_grad()
    loss.backward()
    optimizer_ft.step()
    total_train_loss+=loss
    train_correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
  model_ft.eval()
  for (x,y) in target_val_loader:
    (x,y)=(x.to(device),y.to(device))
    pred=model_ft(x)
    loss=criterion(pred,y)
    # optimizer_ft.zero_grad()
    # loss.backward()
    # optimizer_ft.step()
    total_val_loss+=loss
    val_correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
  
  avg_train_loss=total_train_loss/target_train_steps
  avg_val_loss=total_train_loss/target_val_steps
  train_correct = train_correct/len(target_train_set)
  val_correct = val_correct/len(target_val_set)

  print("Training Loss",avg_train_loss)
  print("Training Accuracy",train_correct)
  print("Val Loss",avg_val_loss)
  print("Val Accuracy",val_correct)



Training Loss tensor(0.0817, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 0.9642857142857143
Val Loss tensor(0.5722, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.35714285714285715
Training Loss tensor(0.0150, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 1.0
Val Loss tensor(0.1047, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.6428571428571429
Training Loss tensor(0.0013, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 1.0
Val Loss tensor(0.0088, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.5714285714285714
Training Loss tensor(0.0008, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 1.0
Val Loss tensor(0.0058, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.5714285714285714
Training Loss tensor(0.0012, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy 1.0
Val Loss tensor(0.0081, device='cuda:0', grad_fn=<DivBackward0>)
Val Accuracy 0.6428571428571429
